In [1]:
def main():
    jsonResult = []
    result = []
    
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"
    
    jsonResult, result, natName, dataEND = getTourismStatsService(nat_cd, ed_cd,
                                                                 nStartYear, nEndYear)
    
    #파일저장 1 : json 파일
    with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w',
             encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, 
                              ensure_ascii = False)
        outfile.write(jsonFile)
    
    #파일저장 2 : csv 파일
    columns = ['입국자국가', '국가코드', '입국연월', '입국자 수']
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND),
                    index = False, encoding = 'cp949')

In [2]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
def getTourismStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json%serviceKey=" + "lcAPXOPlUUuDvgQSWdjYPP0Y29wiY1Q3OH7JsaCzLah0PUiS1tQaE9gYn4sL%2F%2BFWPMWkcigKRLoDh21BnYBLuw%3D%3D"
    
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD" + nat_cd
    parameters += "&ED_CD" + ed_cd
    
    url = service_url + parameters
    responseDecode = getRequestUrl(url)
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [4]:
def getTourismStatsService(nat_cd,ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName = ''
    dataEnd = "{0}{1:0>2}".format(str(nEndYear), str(12))
    isDataEnd = 0
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            if(isDataEnd == 1): break
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)
            if(jsonData['response']['header']['resultMsg'] == 'OK'):
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음... \n 제공되는 통계 데이터는 %s년 %s월까지입니다."
                            %(str(year), str(month-1)))
                    break
                print(json.dumps(jsonData, indent = 4,
                                sort_keys = True, ensure_ascii = False))
                natName = jsonData['response']['body']['items']['items']['natKorNm']
                natName = natName.replace(' ','')
                num = jsonData['response']['body']['items']['items']['num']
                ed = jsonData['response']['body']['items']['items']['ed']
                print('[ %s_%s : %s]' %(natName, yyyymm, num))
                print("--------------------------------------")
                    
                jsonResult.append({'nat_name' : natName, 'nat_cd': nat_c,
                                   'yyyymm' : yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
        return(jsonResult, result, natName,ed, dataEND)

In [22]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey = "lcAPXOPlUUuDvgQSWdjYPP0Y29wiY1Q3OH7JsaCzLah0PUiS1tQaE9gYn4sL%2F%2BFWPMWkcigKRLoDh21BnYBLuw%3D%3D"

#[코드 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
#[코드 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json%serviceKey=" + ServiceKey
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD" + national_code
    parameters += "&ED_CD" + ed_cd
    url = service_url + parameters
    
    retData = getRequestUrl(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)
    
#[코드 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName = ''
    dataEnd = "{0}{1:0>2}".format(str(nEndYear), str(12))
    isDataEnd = 0
    
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            if(isDataEnd == 1): break
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'):  
                if jsonData['response']['body']['items'] == '':
                    isDataEnd = 1
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))
                    break
                    
                print(json.dumps(jsonData, indent = 4,
                                sort_keys = True, ensure_ascii = False))
                natName = jsonData['response']['body']['items']['items']['natKorNm']
                natName = natName.replace(' ','')
                num = jsonData['response']['body']['items']['items']['num']
                ed = jsonData['response']['body']['items']['items']['ed']
                print('[ %s_%s : %s]' %(natName, yyyymm, num))
                print("--------------------------------------")
                    
                jsonResult.append({'nat_name' : natName, 'nat_cd': nat_c, 'yyyymm' : yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
        return(jsonResult, result, natName,ed, dataEND)
    
#[코드 0]
def main():
    jsonResult = []
    result = []
    natName = ''
    
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"
    
    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear)
        
    #파일저장 1 : json 파일
    with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)
    
    #파일저장 2 : csv 파일
    columns = ['입국자국가', '국가코드', '입국연월', '입국자 수']
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND), index = False, encoding = 'cp949')
    

if __name__ == '__main__':
    main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 112
데이터를 몇 년부터 수집할까요? : 2015
데이터를 몇 년까지 수집할까요? : 2020
[2023-04-06 12:10:01.638151] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 394345,
                    "rnum": 1,
                    "ym": 201501
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_201501 : 394345 ]
----------------------------------------------------------------------
[2023-04-06 12:10:01.688799] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관

[2023-04-06 12:10:02.356833] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 521981,
                    "rnum": 1,
                    "ym": 201601
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_201601 : 521981 ]
----------------------------------------------------------------------
[2023-04-06 12:10:02.408837] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                   

[2023-04-06 12:10:03.354752] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 590790,
                    "rnum": 1,
                    "ym": 201702
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_201702 : 590790 ]
----------------------------------------------------------------------
[2023-04-06 12:10:03.419706] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                   

[2023-04-06 12:10:04.333128] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 403413,
                    "rnum": 1,
                    "ym": 201803
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_201803 : 403413 ]
----------------------------------------------------------------------
[2023-04-06 12:10:04.425843] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                   

[2023-04-06 12:10:05.346347] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 493250,
                    "rnum": 1,
                    "ym": 201904
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_201904 : 493250 ]
----------------------------------------------------------------------
[2023-04-06 12:10:05.422173] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                   

[2023-04-06 12:10:06.307151] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 5124,
                    "rnum": 1,
                    "ym": 202005
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 중국_202005 : 5124 ]
----------------------------------------------------------------------
[2023-04-06 12:10:06.378931] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "nu